In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
def unfold (A, mode):
    m, n, d = A.shape

    if(mode == 1):
        folds = []
        for i in range(n):
            folds.append(A[:, i, :])
        # print(np.hstack(folds))
        return(np.array(folds))
    
    elif(mode == 2):
        folds = []
        for i in range(d):
            folds.append(A[:, :, i].T)
        # print(np.hstack(folds))
        return(np.array(folds))
    
    elif(mode == 3):
        folds = []
        for i in range(m):
            folds.append(A[i, :, :].T)
        # print(np.hstack(folds))
        return(np.array(folds))


In [3]:
def tensor_mul (A, v, mode):
    m, n, d = A.shape
    q,_ = v.shape

    if(mode == 1):
        folds = unfold(A, mode=1)
        new_unfold_A = []
        for fold in folds:
            new_unfold_A.append(np.dot(v,fold))
        new_A  = np.empty((q,n,d))
        for i in range(n):
            new_A[:, i, :] = new_unfold_A[i]
        return(new_A)

    elif(mode == 2):
        folds = unfold(A, mode=2)
        new_unfold_A = []
        for fold in folds:
            new_unfold_A.append(np.dot(v,fold))
        new_A  = np.empty((m,q,d))
        for i in range(d):
            new_A[:, :, i] = new_unfold_A[i].T
        return(new_A)

    elif(mode == 3):
        folds = unfold(A, mode=3)
        new_unfold_A = []
        for fold in folds:
            new_unfold_A.append(np.dot(v,fold))
        new_A  = np.empty((m,n,q))
        for i in range(m):
            new_A[i, :, :] = new_unfold_A[i].T
        return(new_A)

In [4]:
def HOSVD(A, is_full = False):
    U = []
    S = np.copy(A)
    if (is_full):
        u1, _, _ = np.linalg.svd(np.hstack(unfold(A, mode=1)), full_matrices=True)
        U.append(u1)
        u2, _, _ = np.linalg.svd(np.hstack(unfold(A, mode=2)), full_matrices=True)
        U.append(u2)
        u3, _, _ = np.linalg.svd(np.hstack(unfold(A, mode=3)), full_matrices=True)
        U.append(u3)
    else:
        u1, _, _ = np.linalg.svd(np.hstack(unfold(A, mode=1)), full_matrices=False)
        U.append(u1)
        u2, _, _ = np.linalg.svd(np.hstack(unfold(A, mode=2)), full_matrices=False)
        U.append(u2)
        u3, _, _ = np.linalg.svd(np.hstack(unfold(A, mode=3)), full_matrices=False)
        U.append(u3)
    S = tensor_mul (S, u1.T, mode=1)
    S = tensor_mul (S, u2.T, mode=2)
    S = tensor_mul (S, u3.T, mode=3)
    return (S, U)

# Test

In [5]:
A_test = np.array([[[1, 0, 2], [1, -1, -1]], 
                 [[0, 1, 3], [2, 2, -1]], 
                 [[4, 5, 0], [1, 2, -2]], 
                ])
v_test = np.array([[-1, 0], [-2, 1], [2, 3]])

tensor_mul(A_test.transpose(1,2,0), v_test, mode=1).transpose(2,0,1)

array([[[-1.,  0., -2.],
        [-1., -1., -5.],
        [ 5., -3.,  1.]],

       [[ 0., -1., -3.],
        [ 2.,  0., -7.],
        [ 6.,  8.,  3.]],

       [[-4., -5.,  0.],
        [-7., -8., -2.],
        [11., 16., -6.]]])

In [6]:
A_test = np.array([[[1, 0], [2, 3]], 
                 [[2, -1], [0, 1]], 
                 [[-3, 1], [-1, -2]], 
                ])
v_test = np.array([[1, 0], [-2, 3], [4, -1]])

tensor_mul(A_test.transpose(1,2,0), v_test, mode=2).transpose(2,0,1)

array([[[  1.,  -2.,   4.],
        [  2.,   5.,   5.]],

       [[  2.,  -7.,   9.],
        [  0.,   3.,  -1.]],

       [[ -3.,   9., -13.],
        [ -1.,  -4.,  -2.]]])

In [7]:
A_test = np.array([[[2, 3, 1], [-1, 0, -2]], 
                 [[2, 4, 1], [-3, 0, 1]], 
                ])
v_test = np.array([[1, 0], [2, -1], [3, 3]])

tensor_mul(A_test.transpose(1,2,0), v_test, mode=3).transpose(2,0,1)

array([[[  2.,   3.,   1.],
        [ -1.,   0.,  -2.]],

       [[  2.,   2.,   1.],
        [  1.,   0.,  -5.]],

       [[ 12.,  21.,   6.],
        [-12.,   0.,  -3.]]])

# Algorithm

In [8]:
import os
import glob
from PIL import Image

cwd = os.getcwd()
input_file = os.path.join(cwd, 'YALE_faces', 'train', '*.gif')
a_person = []
all_person = []
for filename in glob.glob(input_file ):
    img=Image.open(filename)
    np_img = np.array(img)
    np_img = np_img.flatten('F')
    np_img = np_img.reshape(len(np_img),1)
    a_person.append(np_img)
    if (len(a_person) == 10):
        all_person.append(np.hstack(a_person))
        a_person = []

all_person = np.array(all_person)
all_person = all_person.transpose(1, 2, 0)

In [9]:
all_person.shape

(77760, 10, 14)

In [10]:
m, n, l = all_person.shape
S, U = HOSVD(all_person)
HT = U[2].T
q, p = HT.shape

# Slow

In [14]:
########################################
C = tensor_mul (S, U[0], mode=1)
C = tensor_mul (C, U[1], mode=2)
print(C.shape)
########################################


input_test_file = os.path.join(cwd, 'YALE_faces', 'test', '*.gif')
y_true = []
y_predict = []
p_person = []


for filename in  glob.glob(input_test_file ):
    target = filename.removeprefix(os.path.join(cwd, 'YALE_faces', 'test')).removesuffix('.gif')
    if (target[-9:] == "not_exist"):
        y_true.append(1)
    else:
        y_true.append(0)

    img_test=Image.open(filename)
    np_img_test = np.array(img_test)
    np_img_test = np_img_test.flatten('F')
    np_img_test = np_img_test.reshape(-1, 1)

    dict_aes = {}
    dict_errors = {}
    for i in range(n):
        ae, error, _, _= np.linalg.lstsq(C[:, i, :], np_img_test, rcond=None)
        dict_aes[i] = ae
        dict_errors[i] = error[0]

    for mode in dict_errors:
        if (dict_errors[mode] == min(dict_errors.values())):
            check = mode
            break

    flag_not_exist = True
    for i in range(p):
        temp = HT[:, i].reshape(-1,1)
        if (np.linalg.norm(dict_aes[check] - temp) < 0.50):
            if (flag_not_exist):
                y_predict.append(0)
                flag_not_exist = False
                p_person.append((i, target))
            
            p_person.append((i, target))

    if (flag_not_exist):
        y_predict.append(1)

(77760, 10, 14)


In [15]:
print(y_true)
print(y_predict)
print("accuracy_score: ", accuracy_score(y_true, y_predict))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy_score:  0.8


In [16]:
########################################
C = tensor_mul (S, U[0], mode=1)
C = tensor_mul (C, U[1], mode=2)
########################################

input_test_file = os.path.join(cwd, 'YALE_faces', 'test', '*.gif')
y_true = []
y_predict = []
p_person = []

for filename in  glob.glob(input_test_file ):
    target = filename.removeprefix(os.path.join(cwd, 'YALE_faces', 'test')).removesuffix('.gif')
    if (target[-9:] == "not_exist"):
        y_true.append(1)
    else:
        y_true.append(0)

    img_test=Image.open(filename)
    np_img_test = np.array(img_test)
    np_img_test = np_img_test.flatten('F')
    np_img_test = np_img_test.reshape(-1,1)

    flag_not_exist = True
    for i in range(n):
        ae, _, _, _= np.linalg.lstsq(C[:, i, :], np_img_test, rcond=None)
        for j in range(p):
            temp = HT[:, j].reshape(-1,1)
            if (np.linalg.norm(ae - temp) < 0.50):
                if (flag_not_exist):
                    y_predict.append(0)
                    flag_not_exist = False
                    p_person.append((j, target))
                p_person.append((j, target))
    if (flag_not_exist):
        y_predict.append(1)

In [17]:
print(y_true)
print(y_predict)
print("accuracy_score: ", accuracy_score(y_true, y_predict))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy_score:  0.84


# Fast

In [18]:
########################################
Ft = U[0].T 
B = tensor_mul (S, U[1], mode=2)
########################################


input_test_file = os.path.join(cwd, 'YALE_faces', 'test', '*.gif')
y_true = []
y_predict = []
p_person = []

for filename in  glob.glob(input_test_file ):
    target = filename.removeprefix(os.path.join(cwd, 'YALE_faces', 'test')).removesuffix('.gif')
    if (target[-9:] == "not_exist"):
        y_true.append(1)
    else:
        y_true.append(0)

    img_test=Image.open(filename)
    np_img_test = np.array(img_test)
    np_img_test = np_img_test.flatten('F')
    np_img_test = np_img_test.reshape(-1,1)

    dict_aes = {}
    dict_errors = {}
    for i in range(n):
        ae, error, _, _= np.linalg.lstsq(B[:, i, :], Ft @ np_img_test, rcond=None)
        dict_aes[i] = ae
        dict_errors[i] = error[0]

    for mode in dict_errors:
        if (dict_errors[mode] == min(dict_errors.values())):
            check = mode
            break

    flag_not_exist = True
    for i in range(p):
        temp = HT[:, i].reshape(-1,1)
        if (np.linalg.norm(dict_aes[check] - temp) < 0.50):
            if (flag_not_exist):
                y_predict.append(0)
                flag_not_exist = False
                p_person.append((i, target))
            
            p_person.append((i, target))

    if (flag_not_exist):
        y_predict.append(1)

In [19]:
from sklearn.metrics import accuracy_score

print(y_true)
print(y_predict)
print("accuracy_score: ", accuracy_score(y_true, y_predict))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy_score:  0.8


In [20]:
########################################
Ft = U[0].T 
B = tensor_mul (S, U[1], mode=2)
########################################


input_test_file = os.path.join(cwd, 'YALE_faces', 'test', '*.gif')
y_true = []
y_predict = []
p_person = []

for filename in  glob.glob(input_test_file ):
    target = filename.removeprefix(os.path.join(cwd, 'YALE_faces', 'test')).removesuffix('.gif')
    if (target[-9:] == "not_exist"):
        y_true.append(1)
    else:
        y_true.append(0)

    img_test=Image.open(filename)
    np_img_test = np.array(img_test)
    np_img_test = np_img_test.flatten('F')
    np_img_test = np_img_test.reshape(-1,1)

    flag_not_exist = True
    for i in range(n):
        ae, _, _, _= np.linalg.lstsq(B[:, i, :], Ft @ np_img_test, rcond=None)
        for j in range(p):
            temp = HT[:, j].reshape(-1,1)
            if (np.linalg.norm(ae - temp) < 0.50):
                if (flag_not_exist):
                    y_predict.append(0)
                    flag_not_exist = False
                    p_person.append((j, target))
                p_person.append((j, target))
    if (flag_not_exist):
        y_predict.append(1)


In [21]:
print(y_true)
print(y_predict)
print("accuracy_score: ", accuracy_score(y_true, y_predict))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy_score:  0.84
